In [2]:
# Modulos a utilizar
import os
import pandas as pd
import numpy as np
# from google.colab import drive
import pyarrow as pa
import pyarrow.parquet as pq
import pickle

import warnings
warnings.filterwarnings(action= 'ignore')

StatementMeta(, , , Waiting, )

In [ ]:
df = spark.read.format("csv").option("header","true").load("Files/Copy_Google/metadata_sitios/df_unido.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Copy_Google/metadata_sitios/df_unido.csv".
display(df)

In [ ]:
current_directory = os.getcwd()

# Mostrar el directorio actual
print("Estás en el directorio:", current_directory)
desired_directory = '/lakehouse/default/Files
'
os.chdir(desired_directory)
print("Ahora Estás en el directorio:", desired_directory)

StatementMeta(, 0e43904c-45b3-480f-a769-b448af6780b1, 6, Finished, Available)

Estás en el directorio: /lakehouse/default/Files/Copy_Google
Ahora Estás en el directorio: /lakehouse/default/Files


In [ ]:
def verificar_tipo_datos(df):
    '''
    Realiza un análisis de los tipos de datos y la presencia de valores nulos en un DataFrame.

    Esta función toma un DataFrame como entrada y devuelve un resumen que incluye información sobre
    los tipos de datos en cada columna, el porcentaje de valores no nulos y nulos, así como la
    cantidad de valores nulos por columna.

    Parameters:
        df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
        pandas.DataFrame: Un DataFrame que contiene el resumen de cada columna, incluyendo:
        - 'nombre_campo': Nombre de cada columna.
        - 'tipo_datos': Tipos de datos únicos presentes en cada columna.
        - 'no_nulos_%': Porcentaje de valores no nulos en cada columna.
        - 'nulos_%': Porcentaje de valores nulos en cada columna.
        - 'nulos': Cantidad de valores nulos en cada columna.
    '''

    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)

    return df_info

In [ ]:
def informe_dataframe(dataframe: pd.DataFrame) -> None:

    """
    esta funcion obtiene un dataframe, y realiza un informe analizando y explorando algunas caracteristicas del
    dataframe centrandose principalmente en caracteristicas a nivel general de nuestro dataframe y realizando un procesamiento de
    algunos datos obteniendo metricas e informacion

    devuelve un informe que contiene:

    -Dimensiones del DataFrame
    -Numero de datos
    -Filas y Columnas
    -Metricas Generales

    Parameters: data (pandas.DataFrame).

    Returns: None.

    """

    df = dataframe

    print('INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:\n')
    print(f'--Dimensiones del DataFrame--\nFilas: {df.shape[0]}\nColumnas: {df.shape[1]}\n')
    print(f'--Numero de datos--\n{df[df.isna() == False].count().sum()}\n')
    print(f'--Filas y Columnas--\nFilas: muestra de indices-------> {list(df.index)[0:5]}  -----> Desde {list(df.index)[0]}  Hasta {list(df.index)[-1]}\nColumnas: {list(df.columns)}\n')
    # print(f'--Estadisticos preliminares generales--\n{df.describe()}\n')

    return ('~'*50)+'oo'+('~'*50)

In [ ]:
def analisis_estado(estado: str):

  """
  Esta funcion obtiene el nombre de un estado, y llama al archivo .parquet referente al mismo estado obtenido
  y lo transforma en un pandas.DataFrame.

  Luego realiza un analisis obteniendo diversa informacion del dataframe en general y de los datos presentes
  en las columnas en particular imprimiendo sus resultados en pantalla, utilizando las funciones:

  - informe_dataframe(df)
  - verificar_tip_datos(df)

  Parameters: estado (str)
  Returns: None
  """

  #url= f"/content/drive/MyDrive/Proyecto final henry/reviews-estados/review-{estado}/"
  #cantidad_data = len(os.listdir(url))

  url = "/content/drive/MyDrive/Proyecto final henry/merge_estados/"


  df= pd.read_parquet(url + f"df_unido_{estado}.parquet")
  print(f'Nombre del archivo: {estado}.parquet')
  print(f'Estado: {estado}\n')
  print(informe_dataframe(df), end= '\n\n')
  print(verificar_tipo_datos(df))
  print('-'*100)
  print('-'*100)

In [3]:
def merge_reviews():
    ruta_principal = "/lakehouse/default/Files/Google/reviews-estados"

    # Lista para almacenar los DataFrames de cada archivo JSON
    dfs = []

    # Recorrer las subcarpetas en la carpeta principal
    for subdir, _, archivos in os.walk(ruta_principal):
        for archivo in archivos:
            if archivo.endswith(".json"):
                ruta_archivo = os.path.join(subdir, archivo)
                # Leer el archivo JSON en un DataFrame de Pandas
                df = pd.read_json(ruta_archivo, orient='records', lines=True)
                dfs.append(df)

    # Concatenar todos los DataFrames en uno solo
    df_unido = pd.concat(dfs, ignore_index=True)

    # Guardar el DataFrame unificado como un archivo Parquet
    output_parquet = "/lakehouse/default/Files/Copy_Google/Reviews_estados/unificado.parquet"
    df_unido.to_parquet(output_parquet)

    print("Archivos JSON unificados y convertidos a formato Parquet exitosamente.")

StatementMeta(, , , Waiting, )

Archivos JSON unificados y convertidos a formato Parquet exitosamente.


In [6]:
def merge_dfs_final_metadata():

    """
    Esta funcion concatena los archivos .parquet referentes a metadatos de la carpeta merge_metadata y los guarda en un unico archivo .parquet

    Parameters: None
    Returns: None
    """
    url = "/lakehouse/default/Files/Google/metadata-sitios/"
    archivos = os.listdir(url)

    # Lista para almacenar los DataFrames de cada archivo JSON
    dfs = []

    for archivo in archivos:
        if archivo.endswith(".json"):
            ruta_archivo = os.path.join(url, archivo)
            df = pd.read_json(ruta_archivo, orient='records', lines=True)
            dfs.append(df)
    # Concatenar todos los DataFrames en uno solo
    df_unido = pd.concat(dfs, ignore_index=True)
    # Guardar el DataFrame unificado como un archivo .csv con codificación UTF-8
    output_csv = "/lakehouse/default/Files/Copy_Google/metadata_sitios/df_unido.csv"
    df_unido.to_csv(output_csv, encoding='utf-8', index=False)  # Especificamos la codificación y evitamos guardar el índice
    print(df_unido.columns)
    print("Archivos JSON unificados y convertidos a formato CSV exitosamente.")


StatementMeta(, 47880ad1-74d0-4872-ba3c-0a3fb2a238d0, 7, Finished, Available)

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')
Archivos JSON unificados y convertidos a formato CSV exitosamente.


## Nuestros datos son extraidos de dos fuentes principales:

# GOOGLE MAPS

## metadata-sitios

Uniremos los archivos en 3 archivos parquet que abarquen cada uno 4 archivos y uno 3


In [23]:
merge_dfs_final_metadata()
merge_reviews()

StatementMeta(, 3e0d064a-1d72-4f78-98ba-c9dac952f363, 25, Finished, Available)

               name                                            address  \
0   Porter Pharmacy  Porter Pharmacy, 129 N Second St, Cochran, GA ...   
1      City Textile  City Textile, 3001 E Pico Blvd, Los Angeles, C...   
2      San Soo Dang  San Soo Dang, 761 S Vermont Ave, Los Angeles, ...   
3      Nova Fabrics  Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...   
4  Nobel Textile Co  Nobel Textile Co, 719 E 9th St, Los Angeles, C...   

                                 gmap_id description   latitude   longitude  \
0  0x88f16e41928ff687:0x883dad4fd048e8f8        None  32.388300  -83.357100   
1  0x80c2c98c0e3c16fd:0x29ec8a728764fdf9        None  34.018891 -118.215290   
2  0x80c2c778e3b73d33:0xbdc58662a4a97d49        None  34.058092 -118.292130   
3   0x80c2c89923b27a41:0x32041559418d447        None  34.023669 -118.232930   
4  0x80c2c632f933b073:0xc31785961fe826a6        None  34.036694 -118.249421   

              category  avg_rating  num_of_reviews price  \
0           [Pharmac

# YELP

### Bussines

In [ ]:
#url de fichero bussines.pkl
bussi_yelp= "/content/drive/MyDrive/Proyecto final henry/Yelp/business.pkl"

In [ ]:
# Abrir el archivo en modo de lectura binaria
with open(bussi_yelp, 'rb') as f:
    # Cargar el objeto desde el archivo .pkl
    objeto = pickle.load(f)

In [ ]:
# transformamos la data en un dataframe
df_bussi = pd.DataFrame(objeto)